In [3]:
import slate3k as slate
import re
import PyPDF2
import requests
import os
import csv
import pathlib

In [4]:
# Using PyPDF2
#able to retrieve hyperlinks for URLs
PDFFile = open("target.pdf",'rb')
PDF = PyPDF2.PdfFileReader(PDFFile)

In [5]:
# Using slate for text  mining
# Slate is able to fully extract text compared to PyPDF2
with open("Target.pdf","rb") as f:
    extracted_text = slate.PDF(f)

In [6]:
# API URL and token for IP Info APs
api_URL = "https://ipinfo.io/"
api_token = '5c213747c11f8c'

In [26]:
# Convert Dict To CSV and outputs a file at current directory
def fromDictToCSV(headings,myDictData):

    #remove file if exists
    if os.path.exists("ipdesc.csv"):
        os.remove("ipdesc.csv")

    
    myFilePath = os.path.join('ipdesc.csv')
    with open(myFilePath,'w+', newline='') as myCSVFile:
        csvWriter = csv.DictWriter(myCSVFile, fieldnames=headings, dialect='excel', quoting=csv.QUOTE_NONNUMERIC)
        csvWriter.writeheader()
        for data in myDictData:
            csvWriter.writerow(data)

    return

In [8]:
# Retrieve ASN by using API calls
def retrieveASN(listOfIP):
    listOfInfo = []
    for ip in listOfIP:
        # defining a params dict for the parameters to be sent to the API
        PARAMS = {'token':api_token}
        r = requests.get(url = api_URL+ip,params = PARAMS)
        data = r.json()
        listOfInfo.append(data)
        
    return listOfInfo

In [9]:
# Extract URLS from Hyperlinks in PDF Using PyPDF2
def getUrlsFromPDF(PDF):
    pages = PDF.getNumPages()
    key = '/Annots'
    uri = '/URI'
    ank = '/A'
    urls = []
    for page in range(pages):

        pageSliced = PDF.getPage(page)
        pageObject = pageSliced.getObject()
        if key in pageObject.keys():
            ann = pageObject[key]
            for a in ann:
                u = a.getObject()
                if uri in u[ank].keys():
                    urls.append(u[ank][uri])
    return urls

In [11]:
def getAllIPsAndHashes(extractText):
    listOfIP = []
    hashes = []
    for lines in extracted_text:
        # listOfURLs = re.findall(URL_REGEX,lines) + listOfURLs
        lines = lines.replace('[','')
        lines =lines.replace(']','')
        
        #Regex for IP address
        listOfIP = re.findall( r'[0-9]+(?:\.[0-9]+){3}', lines) + listOfIP
        
        #Converting lines to lower cases and using regex to match hashes
        hashes = re.findall(r'\b[0-9a-f]{40}\b',lines.lower()) + hashes


        #print(lines)
    return [listOfIP,hashes]

In [12]:
listOfIPandHashes = getAllIPsAndHashes(extracted_text)
listOfIP = listOfIPandHashes[0]
listOfHashes = listOfIPandHashes[1]
listOfInfo = retrieveASN(listOfIP)

In [27]:
fromDictToCSV(list((listOfInfo[0].keys())),listOfInfo)